In [140]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
    
parent_dir = os.path.abspath(r'C:\Users\Clayg\OneDrive\Desktop\College\Fall 25 classes\Machine Learning\CS-3120-Project')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)


from DataPipeline import pipeline
from MappingPipeline import mappingPipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
folder_paths= [r"C:\Users\Clayg\OneDrive\Desktop\College\Fall 25 classes\Machine Learning\RawData\FARS2020NationalCSV",
               r"c:\Users\Clayg\OneDrive\Desktop\College\Fall 25 classes\Machine Learning\RawData\FARS2021NationalCSV",
               r"c:\Users\Clayg\OneDrive\Desktop\College\Fall 25 classes\Machine Learning\RawData\FARS2022NationalCSV"]


In [142]:
testPipeline = pipeline(folder_paths)

In [143]:
directory = testPipeline.df_list

df2020 = directory[2020]
df2021 = directory[2021]
df2022 = directory[2022]

In [144]:
df = pd.concat([df2020, df2021, df2022])

In [145]:
print(df.columns.to_list())

['ST_CASE', 'VEH_NO', 'OWNER', 'MOD_YEAR', 'VPICBODYCLASS', 'GVWR_FROM', 'GVWR_TO', 'J_KNIFE', 'TRAV_SP', 'DR_DRINK', 'L_STATUS', 'PREV_SPD', 'PREV_OTH', 'SPEEDREL', 'VE_TOTAL', 'VE_FORMS', 'HOUR', 'LGT_COND', 'RUR_URB', 'FUNC_SYS', 'YEAR', 'HARM_EV', 'MAN_COLL', 'RELJCT2', 'TYP_INT', 'REL_ROAD', 'WEATHER', 'VNUMBER2']


In [146]:
mapping = mappingPipeline()

df = mapping.assign_mappings(df)

In [147]:
df = df.drop_duplicates()

In [148]:
print(np.sort(df['MOD_YEAR'].unique().tolist()))

[   0 1951 1954 1955 1960 1961 1962 1965 1966 1967 1968 1969 1970 1971
 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985
 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


## Feature Engineering

Age of veh:

In [149]:
def ageCalc(row):
    if row['MOD_YEAR'] == 0:
        return 999
    else:
        age = row['YEAR']-row['MOD_YEAR']
        return age 

In [150]:
df['VEH_AGE'] = df.apply(ageCalc, axis=1)

ISMOTO (is a motorcycle)

In [151]:
motorcycle_codes = [12,69,80,81,82,83,84,87,88,90,94,109,110,113,114,125,996]
df['ISMOTO'] = df['VPICBODYCLASS'].isin(motorcycle_codes)

Speed of other vehicle (if vehicle 1 is a motorcyle)

**Gemini assisted in suggesting and creating the indexing code. It also created the try/except block for code safety**

In [152]:
def otherVehSpeed(row):

    if row["ISMOTO"]:
        veh_num = row["VNUMBER2"]
        year = row["YEAR"]
        case_num = row['ST_CASE']
        try:
            # This is the line that might fail
            other_veh_speed = df_indexed.loc[(year, case_num, veh_num), "TRAV_SP"]

            if isinstance(other_veh_speed, pd.Series):
                return other_veh_speed.iloc[0]
            else: return other_veh_speed
            
            
        except KeyError:
            # This runs if the .loc[] lookup fails to find a match
            return np.nan  # Or 888, 999, or another "not found" code

    else: return 999

        

In [153]:
df_indexed = df.set_index(["YEAR", "ST_CASE", "VEH_NO"])
df_indexed = df_indexed.sort_index()

In [154]:
df['OTHER_VEH_SP'] = df.apply(otherVehSpeed, 1)

Drop rows where ISMOTO is false, leaving us only with motorcycle data

In [155]:
df = df[df["ISMOTO"]]

We're going to drop all the columns that have helped us feature engineer but are no longer useful for model building

In [156]:
print(df.columns.tolist())

['ST_CASE', 'VEH_NO', 'OWNER', 'MOD_YEAR', 'VPICBODYCLASS', 'GVWR_FROM', 'GVWR_TO', 'J_KNIFE', 'TRAV_SP', 'DR_DRINK', 'L_STATUS', 'PREV_SPD', 'PREV_OTH', 'SPEEDREL', 'VE_TOTAL', 'VE_FORMS', 'HOUR', 'LGT_COND', 'RUR_URB', 'FUNC_SYS', 'YEAR', 'HARM_EV', 'MAN_COLL', 'RELJCT2', 'TYP_INT', 'REL_ROAD', 'WEATHER', 'VNUMBER2', 'VEH_AGE', 'ISMOTO', 'OTHER_VEH_SP']


In [161]:
cols_to_drop = ['MOD_YEAR', 'VPICBODYCLASS', 'VNUMBER2', 'ISMOTO']
df = df.drop(columns=cols_to_drop)

In [162]:
print(df.columns.tolist())

['ST_CASE', 'VEH_NO', 'OWNER', 'GVWR_FROM', 'GVWR_TO', 'J_KNIFE', 'TRAV_SP', 'DR_DRINK', 'L_STATUS', 'PREV_SPD', 'PREV_OTH', 'SPEEDREL', 'VE_TOTAL', 'VE_FORMS', 'HOUR', 'LGT_COND', 'RUR_URB', 'FUNC_SYS', 'YEAR', 'HARM_EV', 'MAN_COLL', 'RELJCT2', 'TYP_INT', 'REL_ROAD', 'WEATHER', 'VEH_AGE', 'OTHER_VEH_SP']


In [163]:
df.head()

,ST_CASE,VEH_NO,OWNER,GVWR_FROM,GVWR_TO,J_KNIFE,TRAV_SP,DR_DRINK,L_STATUS,PREV_SPD,...,FUNC_SYS,YEAR,HARM_EV,MAN_COLL,RELJCT2,TYP_INT,REL_ROAD,WEATHER,VEH_AGE,OTHER_VEH_SP
0,10087,1,2,11,11,0,60,1,1,2,...,3,2020,1.0,9,1,1,3,4,22.0,NaN
2,10114,1,1,11,11,0,0,0,1,0,...,3,2020,4.0,8,1,1,1,4,8.0,NaN
3,10114,1,1,11,11,0,0,0,1,0,...,3,2020,4.0,8,1,1,1,4,8.0,30.0
7,10201,2,2,11,11,0,75,0,1,1,...,3,2020,4.0,6,2,3,1,1,20.0,0.0
8,10201,2,2,11,11,0,75,0,1,1,...,3,2020,4.0,6,2,3,1,1,20.0,NaN
